In [1]:
import pandas as pd
import numpy as np

In [56]:
countries={'Albania': 'AL', 'Armenia': 'AM', 'Austria': 'AT', 'Azerbaijan': 'AZ', 'Belarus': 'BY','Belgium': 'BE', 'Bosnia and Herz.': 'BA', 'Bulgaria': 'BG', 'Croatia': 'HR', 'Cyprus': 'CY', 'Switzerland': 'CH', 'Czech Republic': 'CZ', 'Germany': 'DE', 'Georgia': 'GE', 'Denmark': 'DK', 'Estonia': 'EE', 'Spain': 'ES', 'Finland': 'FI', 'France': 'FR', 'United Kingdom': 'UK', 'Greece': 'GR', 'Hungary': 'HU', 'Iceland': 'IS', 'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Luxembourg': 'LU', 'Latvia': 'LV', 'Montenegro': 'ME', 'North Macedonia': 'MK', 'Netherlands': 'NL', 'Norway': 'NO', 'Poland': 'PL', 'Portugal': 'PT', 'Romania': 'RO', 'Serbia': 'RS', 'Sweden': 'SE', 'Slovenia': 'SI', 'Slovakia': 'SK', 'Kosovo': 'XK', 'Turkey': 'TR', 'Ukraine': 'UA', 'Russia': 'RU',  'Malta': 'MT', 'Moldova': 'MD'}

# Preparing Generation Data

## Getting the generation data of all countries into a single dataframe

In [18]:
temp=pd.read_csv('../Data Sources/ENTSO-E/Load/Germany.csv')

In [43]:
country_data=pd.DataFrame()
country_data['Day']=temp['Time (CET)'].str[:2]
country_data['Month']=temp['Time (CET)'].str[3:5]
country_data['Year']=temp['Time (CET)'].str[6:10]
country_data['Time']=temp['Time (CET)'].str[11:16]+' - '+temp['Time (CET)'].str[29:35]

In [57]:
for country,abbr in countries.items():
    temp=pd.read_csv(f'../Data Sources/ENTSO-E/Load/{country}.csv')
    country_data[f'{abbr}']=temp[f'Actual Total Load [MW] - {country} ({abbr})']

In [59]:
country_data.head()

,Day,Month,Year,Time,AL,AM,AT,AZ,BY,BE,...,RS,SE,SI,SK,XK,TR,UA,RU,MT,MD
0,01,01,2019,00:00 - 00:15,NaN,n/e,6139.0,n/e,n/e,9122.0,...,5290.0,14597.0,1164.0,2835.0,n/e,n/e,16997.0,n/e,n/e,n/e
1,01,01,2019,00:15 - 00:30,NaN,n/e,6108.0,n/e,n/e,9060.0,...,5282.0,14375.0,1129.0,2744.0,n/e,n/e,16726.0,n/e,n/e,n/e
2,01,01,2019,00:30 - 00:45,NaN,n/e,6054.0,n/e,n/e,8964.0,...,5067.0,14153.0,1084.0,2679.0,n/e,n/e,16572.0,n/e,n/e,n/e
3,01,01,2019,00:45 - 01:00,NaN,n/e,5999.0,n/e,n/e,8747.0,...,4739.0,13896.0,1038.0,2608.0,n/e,n/e,16449.0,n/e,n/e,n/e
4,01,01,2019,01:00 - 01:15,NaN,n/e,5976.0,n/e,n/e,8729.0,...,4445.0,13955.0,1018.0,2602.0,n/e,n/e,16300.0,n/e,n/e,n/e
